# Testing data prepared to predict

Strategy: 
- collect an arbitrary data period that contains (begin, end).
- the last value is end. a new_begin is calculated in data_manner.__add_period
- prepare data with build_test(). this automatically removes the last input_window_size from the data to build x, which is the array we will input to predict
- predicts
- get an array of predictions y_hat from new_begin+7 to end
- get the last (end-begin) values from y_hat. this is the forecast from begin to end

In [1]:
configure_json = open('../doc/configure.json', 'r')

import json
#print(json.dumps(json.load(configure_json), indent=4))

In [2]:
import sys
sys.path.append("../src")

import configs_manner

# print("Model infos: \n", configs_manner.model_infos)
# print("\n")
# print("Models path: \n", configs_manner.model_path)
# print("\n")
# print("Data path: \n", configs_manner.data_path)

In [200]:
# import the data_manner.py file. (taking into account that you are in src/ path)
import data_manner
import predictor_manner
from importlib import reload
reload(data_manner)
reload(predictor_manner)

import numpy as np
import datetime

np.set_printoptions(precision=4, suppress=True)

# creating the DataConstructor instance
data_constructor_train = data_manner.DataConstructor()
data_constructor = data_manner.DataConstructor(True)

model_id = "2f36b1b8-c0d1-11ec-9f78-132ad413b391"
# specif code to the remote repository data.
repo = "p971074907"
# coutry and state acronym splited by a ":"
path = "brl:ce"
# columns (or features) to be extracted from the database, each one splited by a ":"
feature = "date:newDeaths:"
# start date for the data request.
begin = "2022-04-10"
# finish date for the data request.
end = "2022-04-30"

DATE_FORMAT = "%Y-%m-%d"

# collect data from the remote repository.
collected_data_train = data_constructor_train.collect_dataframe(path, repo, feature, begin, end)
collected_data = data_constructor.collect_dataframe(path, repo, feature, begin, end)

print("DATA FOR TRAINING", "period:", begin, "-", end)
print(collected_data_train)
print("\n")
print("DATA FOR PREDICTING", "period after add_period:", data_constructor.new_first_day.strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
print("DATA FOR PREDICTING", "period after removing ma buffer:", (data_constructor.new_first_day+datetime.timedelta(days=7)).strftime(DATE_FORMAT), "-", data_constructor.new_last_day.strftime(DATE_FORMAT))
print(collected_data)

predictor = predictor_manner.PredictorConstructor("2f36b1b8-c0d1-11ec-9f78-132ad413b391", path, repo, feature, begin, end)
y_hat = predictor.predict()
print("\n")
print("DATA X prepared by predictor:")
print(predictor.data_X.flatten())
print(predictor.data_X.flatten().shape)
print("\n")
print("PREDICTION using data_X:")
print(y_hat)
print(y_hat.flatten().shape)

DATA FOR TRAINING period: 2022-04-10 - 2022-04-30
[array([3.    , 3.    , 3.    , 3.    , 3.    , 3.    , 3.    , 4.4286,
       4.8571, 5.    , 4.1429, 4.2857, 4.1429, 5.4286, 4.    , 3.2857,
       3.4286, 3.4286, 4.1429, 4.7143, 4.4286])]


DATA FOR PREDICTING period after add_period: 2022-03-26 - 2022-04-30
DATA FOR PREDICTING period after removing ma buffer: 2022-04-02 - 2022-04-30
[array([ 8.1429,  9.1429,  9.8571,  8.4286,  9.5714, 10.7143,  7.1429,
        8.7143,  6.4286,  6.    ,  5.8571,  4.7143,  3.8571,  4.5714,
        3.    ,  4.4286,  4.8571,  5.    ,  4.1429,  4.2857,  4.1429,
        5.4286,  4.    ,  3.2857,  3.4286,  3.4286,  4.1429,  4.7143,
        4.4286])]


DATA X prepared by predictor:
[ 9.1429  9.8571  8.4286  9.5714 10.7143  7.1429  8.7143  6.4286  6.
  5.8571  4.7143  3.8571  4.5714  3.      4.4286  4.8571  5.      4.1429
  4.2857  4.1429  5.4286]
(21,)


PREDICTION using data_X:
[8.3098 8.4021 7.6789 7.49   7.5004 7.441  7.6518 3.2084 2.7639 2.2936
 2.3412

In [199]:
y_hat_weboutput = predictor.predictions_to_weboutput(y_hat)
y_hat_weboutput

[{'date': '2022-04-10', 'prediction': '8.309764'},
 {'date': '2022-04-11', 'prediction': '8.402149'},
 {'date': '2022-04-12', 'prediction': '7.6789246'},
 {'date': '2022-04-13', 'prediction': '7.4900208'},
 {'date': '2022-04-14', 'prediction': '7.500409'},
 {'date': '2022-04-15', 'prediction': '7.440977'},
 {'date': '2022-04-16', 'prediction': '7.651844'},
 {'date': '2022-04-17', 'prediction': '3.2083983'},
 {'date': '2022-04-18', 'prediction': '2.7639148'},
 {'date': '2022-04-19', 'prediction': '2.2935824'},
 {'date': '2022-04-20', 'prediction': '2.3411655'},
 {'date': '2022-04-21', 'prediction': '2.940135'},
 {'date': '2022-04-22', 'prediction': '3.6618717'},
 {'date': '2022-04-23', 'prediction': '4.4354568'},
 {'date': '2022-04-24', 'prediction': '5.2859464'},
 {'date': '2022-04-25', 'prediction': '5.5660853'},
 {'date': '2022-04-26', 'prediction': '5.5511637'},
 {'date': '2022-04-27', 'prediction': '5.6642776'},
 {'date': '2022-04-28', 'prediction': '5.8555365'},
 {'date': '2022-04

In [62]:
import numpy as np
import matplotlib.pyplot as plt

def uniqueish_color():
    """There're better ways to generate unique colors, but this isn't awful."""
    return plt.cm.gist_ncar(np.random.random())

def plot_weeks(x_predict):
    size = x_predict.x.flatten().shape[0]
    x_axis = list(range(size))

    fig, ax = plt.subplots()

    xy = np.vstack((x_axis, x_predict.x.flatten()))

    for i in range(xy.shape[1]//7):
        if i == 0:
            begin = 0
            end = (i+1)*7
        else:
            begin = (i*7)-1
            end = (i+1)*7
        ax.plot(xy[0][begin:end], xy[1][begin:end], color=uniqueish_color())
    plt.show()